In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tangent_boundaries as tb
from math import pi

def staircase_function(energy, spectrum, nbins = 41):
    '''Staircase function.'''
    return np.where(np.sort(np.append(spectrum, energy))==energy)[0][0]

In [ ]:
thetas = [0,1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]

i = -1 ### selected value of theta
Nx = 199
Ny = 145
#nbands = int((Nx*Ny)/100*16)
nbands = 1156

parameters = dict(
    Nx = Nx,
    Ny = Ny,
    B1 = 0, # no magnetic field
    N1 = 0, #
    d1 = 0, # These are irrelevant for B1 = 0
    N2 = 0, #
    potential = lambda x,y:0.0*np.random.rand(Ny,Nx),
    mass = lambda x,y:0*x,
    disorder = 0,
    theta = -(pi/2)*(thetas[i]/100),
)

In [ ]:
#path = '/home/donisvelaa/data1/tangent_boundaries/final/'
name = 'ellipse_spectrum'
#spectrum_ellipse = np.load(path+name+'_Nx'+str(Nx)+'_Ny'+str(Ny)+'_theta'+str(theta)+'.npy', allow_pickle=True)
spectrum_ellipse = np.load(name+'_Nx'+str(parameters['Nx'])+'_Ny'+str(parameters['Ny'])+'_theta'+str(thetas[i])+'_nbands'+str(nbands)+'.npy', allow_pickle=True)
name = 'ellipse_states'
states_ellipse = np.load(name+'_Nx'+str(parameters['Nx'])+'_Ny'+str(parameters['Ny'])+'_theta'+str(thetas[i])+'_nbands'+str(nbands)+'.npy', allow_pickle=True)

In [ ]:
nbins = 61
ellipse_DOS, bin_edges_ellipse_DOS = np.histogram(spectrum_ellipse, bins = nbins, range = (-max(spectrum_ellipse),max(spectrum_ellipse)), density = False)

x = (bin_edges_ellipse_DOS[1:]+bin_edges_ellipse_DOS[:-1])/2
dx = (bin_edges_ellipse_DOS[1]-bin_edges_ellipse_DOS[0])/2

fig_ellipse_DOS = plt.figure(figsize = (7,7))
ax_ellipse_DOS = fig_ellipse_DOS.add_subplot(111)
ax_ellipse_DOS.bar(x,ellipse_DOS,width = 2*dx)
ax_ellipse_DOS.set_ylabel(r"$\rho$ $units$",fontsize = 18)
ax_ellipse_DOS.set_xlabel(r"$E$ $units$",fontsize = 18)
#ax_ellipse_DOS.set_ylim(0,2*(Nx+Ny))


for i in range(nbands):
    to_plot = states_ellipse[i,:,:,:]

    fig = plt.figure(figsize = (3,3))
    ax = fig.add_subplot(111)
    
    ax.imshow(np.sum(np.abs(to_plot)**2,axis = 0),vmin = 0, origin= 'lower')
    #ax.imshow(np.abs(to_plot[1])**2,vmin = 0,cmap = 'cividis') This would plot only spin down probability density
    formatted_number = f"{spectrum_ellipse[i]:.4e}"
    plt.text(2, 8,rf'$E ={formatted_number}$ state' +str(i) , fontsize=10, bbox=dict(facecolor='yellow', alpha=0.5))
    
    ax.set_ylabel(r"$y$ $units$",fontsize = 12)
    ax.set_xlabel(r"$x$ $units$",fontsize = 12)
    
    
    fig.show()

In [ ]:
#energies_ellipse = np.linspace(-max(spectrum_ellipse), max(spectrum_ellipse),1000)
energies_ellipse = np.linspace(-0.3, 0.3,1000)
staircase_ellipse = np.zeros(len(energies_ellipse))
for i, e in enumerate(energies_ellipse):
    staircase_ellipse[i] = staircase_function(e,spectrum_ellipse)

area = (parameters['Nx']//2)*(parameters['Ny']//2)*pi

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.plot(energies_ellipse, staircase_ellipse-staircase_ellipse[len(staircase_ellipse)//2]*np.ones(len(staircase_ellipse)))
ax.plot(energies_ellipse, area*energies_ellipse**2/4/pi)
# ax.axhline(0)
# ax.axhline(0)
# ax.set_xlim(-0.25,0.25)
# ax.set_ylim(-25,25)

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.plot(area*energies_ellipse**2/4/pi, staircase_ellipse-staircase_ellipse[len(staircase_ellipse)//2]*np.ones(len(staircase_ellipse)))
ax.axline((0,0), slope = 1,ls = '--', c = 'r')
#ax.set_xlim(0,160)
#ax.set_ylim(-150,150)

In [ ]:
def spacing_ratios_distribution(spectrum, nbins = 41):
    
    spacings = (np.sort(spectrum)-np.roll(np.sort(spectrum),1))[1:]
    ratios = (spacings/np.roll(spacings,1))[1:]
    ratios = np.where(ratios<1, ratios, 1/ratios)
    histogram, bin_edges = np.histogram(ratios, range = (0,1), bins = nbins, density = True )
    
    return histogram, bin_edges

nbins_spacing = 15
#ratios_distribution_ellipse, bin_edges_ellipse = spacing_ratios_distribution(spectrum_ellipse, nbins = nbins_spacing)

ratios_distribution_ellipse, bin_edges_ellipse = spacing_ratios_distribution(spectrum_ellipse[:len(spectrum_ellipse)//4], nbins = nbins_spacing)

x_ellipse = (bin_edges_ellipse[1:]+bin_edges_ellipse[:-1])/2
dx_ellipse = (bin_edges_ellipse[1]-bin_edges_ellipse[0])
analytical_gue = (81*np.sqrt(3)/(2*pi))*(x_ellipse+x_ellipse**2)**2/(1+x_ellipse+x_ellipse**2)**4

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.bar(x_ellipse,ratios_distribution_ellipse,width = dx_ellipse)
ax.plot(x_ellipse,analytical_gue, c = 'orange')
ax.set_ylabel(r"$P(r)$ $units$",fontsize = 18)
ax.set_xlabel(r"$r$ $units$",fontsize = 18)

In [ ]:
thetas = [0,1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
i = -1 ### selected value of theta
parameters['Nx'] =  int(np.round(np.sqrt(pi*Nx*Ny/4))) # We want the square to have the same area as the ellipse
parameters['Ny'] =  int(np.round(np.sqrt(pi*Nx*Ny/4)))


#path = '/home/donisvelaa/data1/tangent_boundaries/final/'
name = 'square_spectrum'
#spectrum_square = np.load(path+name+'_Nx'+str(Nx)+'_Ny'+str(Ny)+'_theta'+str(theta)+'.npy', allow_pickle=True)
spectrum_square = np.load(name+'_Nx'+str(parameters['Nx'])+'_Ny'+str(parameters['Ny'])+'_theta'+str(thetas[i])+'_nbands'+str(nbands)+'.npy', allow_pickle=True)
name = 'square_states'
states_square = np.load(name+'_Nx'+str(parameters['Nx'])+'_Ny'+str(parameters['Ny'])+'_theta'+str(thetas[i])+'_nbands'+str(nbands)+'.npy', allow_pickle=True)

In [ ]:
nbins = 61
square_DOS, bin_edges_square_DOS = np.histogram(spectrum_square, bins = nbins, range = (-max(spectrum_square),max(spectrum_square)), density = False)

x = (bin_edges_square_DOS[1:]+bin_edges_square_DOS[:-1])/2
dx = (bin_edges_square_DOS[1]-bin_edges_square_DOS[0])/2

fig_square_DOS = plt.figure(figsize = (7,7))
ax_square_DOS = fig_square_DOS.add_subplot(111)
ax_square_DOS.bar(x,square_DOS,width = 2*dx)
ax_square_DOS.set_ylabel(r"$\rho$ $units$",fontsize = 18)
ax_square_DOS.set_xlabel(r"$E$ $units$",fontsize = 18)
#ax_square_DOS.set_ylim(0,2*(Nx+Ny))


for i in range(nbands):
    to_plot = states_square[i,:,:,:]

    fig = plt.figure(figsize = (3,3))
    ax = fig.add_subplot(111)
    
    ax.imshow(np.sum(np.abs(to_plot)**2,axis = 0),vmin = 0, origin = 'lower')
    #ax.imshow(np.abs(to_plot[1])**2,vmin = 0,cmap = 'cividis') This would plot only spin down probability density
    formatted_number = f"{spectrum_square[i]:.4e}"
    plt.text(2, 8,rf'$E ={formatted_number}$' , fontsize=10, bbox=dict(facecolor='yellow', alpha=0.5))
    
    ax.set_ylabel(r"$y$ $units$",fontsize = 12)
    ax.set_xlabel(r"$x$ $units$",fontsize = 12)
    
    
    fig.show()

In [ ]:
#energies_square = np.linspace(-max(spectrum_square), max(spectrum_square),1000)
energies_square = np.linspace(-0.3, 0.3,1000)
staircase_square = np.zeros(len(energies_square))
for i, e in enumerate(energies_square):
    staircase_square[i] = staircase_function(e,spectrum_square)

area = (parameters['Nx'])*(parameters['Ny'])

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.plot(energies_square, staircase_square-staircase_square[len(staircase_square)//2]*np.ones(len(staircase_square)))
ax.plot(energies_square, area*energies_square**2/4/pi)
# ax.axhline(0)
# ax.axhline(0)
# ax.set_xlim(-0.25,0.25)
# ax.set_ylim(-25,25)

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.plot(area*energies_square**2/4/pi, staircase_square-staircase_square[len(staircase_square)//2]*np.ones(len(staircase_square)))
ax.axline((0,0), slope = 1,ls = '--', c = 'r')
#ax.set_xlim(0,160)
#ax.set_ylim(-150,150)

In [ ]:
nbins_spacing = 51
ratios_distribution_square, bin_edges_square = spacing_ratios_distribution(spectrum_square, nbins = nbins_spacing)


x_square = (bin_edges_square[1:]+bin_edges_square[:-1])/2
dx_square = (bin_edges_square[1]-bin_edges_square[0])

analytical_poisson = 2/(1+x_square)**2

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.bar(x_square,ratios_distribution_square,width = dx_square)
ax.plot(x_square,analytical_poisson, c = 'orange')
ax.set_ylabel(r"$P(r)$ $units$",fontsize = 18)
ax.set_xlabel(r"$r$ $units$",fontsize = 18)

In [ ]:
thetas = [0,1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]

i = -1 ### selected value of theta
Nx = 99
Ny = 73
nbands = int((Nx*Ny)/100*16)

parameters = dict(
    Nx = Nx,
    Ny = Ny,
    B1 = 0, # no magnetic field
    N1 = 0, #
    d1 = 0, # These are irrelevant for B1 = 0
    N2 = 0, #
    potential = lambda x,y:0.0*np.random.rand(Ny,Nx),
    mass = lambda x,y:0*x,
    disorder = 0,
    theta = -(pi/2)*(thetas[i]/100),
)

In [ ]:
#path = '/home/donisvelaa/data1/tangent_boundaries/final/'
name = 'stadium_spectrum'
#spectrum_ellipse = np.load(path+name+'_Nx'+str(Nx)+'_Ny'+str(Ny)+'_theta'+str(theta)+'.npy', allow_pickle=True)
spectrum_stadium = np.load(name+'_Nx'+str(parameters['Nx'])+'_Ny'+str(parameters['Ny'])+'_theta'+str(thetas[i])+'_nbands'+str(nbands)+'.npy', allow_pickle=True)
name = 'stadium_states'
states_stadium = np.load(name+'_Nx'+str(parameters['Nx'])+'_Ny'+str(parameters['Ny'])+'_theta'+str(thetas[i])+'_nbands'+str(nbands)+'.npy', allow_pickle=True)

In [ ]:
nbins = 61
stadium_DOS, bin_edges_stadium_DOS = np.histogram(spectrum_stadium, bins = nbins, range = (-max(spectrum_stadium),max(spectrum_stadium)), density = False)

x = (bin_edges_stadium_DOS[1:]+bin_edges_stadium_DOS[:-1])/2
dx = (bin_edges_stadium_DOS[1]-bin_edges_stadium_DOS[0])/2

fig_stadium_DOS = plt.figure(figsize = (7,7))
ax_stadium_DOS = fig_stadium_DOS.add_subplot(111)
ax_stadium_DOS.bar(x,stadium_DOS,width = 2*dx)
ax_stadium_DOS.set_ylabel(r"$\rho$ $units$",fontsize = 18)
ax_stadium_DOS.set_xlabel(r"$E$ $units$",fontsize = 18)
#ax_stadium_DOS.set_ylim(0,2*(Nx+Ny))


for i in range(nbands):
    to_plot = states_stadium[i,:,:,:]

    fig = plt.figure(figsize = (3,3))
    ax = fig.add_subplot(111)
    
    ax.imshow(np.sum(np.abs(to_plot)**2,axis = 0),vmin = 0, origin= 'lower')
    #ax.imshow(np.abs(to_plot[1])**2,vmin = 0,cmap = 'cividis') This would plot only spin down probability density
    formatted_number = f"{spectrum_stadium[i]:.4e}"
    plt.text(2, 8,rf'$E ={formatted_number}$ state' +str(i) , fontsize=10, bbox=dict(facecolor='yellow', alpha=0.5))
    
    ax.set_ylabel(r"$y$ $units$",fontsize = 12)
    ax.set_xlabel(r"$x$ $units$",fontsize = 12)
    
    
    fig.show()

In [ ]:
#energies_stadium = np.linspace(-max(spectrum_stadium), max(spectrum_stadium),1000)
energies_stadium = np.linspace(-0.3, 0.3,1000)
staircase_stadium = np.zeros(len(energies_stadium))
for i, e in enumerate(energies_stadium):
    staircase_stadium[i] = staircase_function(e,spectrum_stadium)

area = pi*(parameters['Ny']//2-1)**2 + 4*(parameters['Ny']//2-1)*(parameters['Nx']//2-1-parameters['Ny']//2-1)

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.plot(energies_stadium, staircase_stadium-staircase_stadium[len(staircase_stadium)//2]*np.ones(len(staircase_stadium)))
ax.plot(energies_stadium, area*energies_stadium**2/4/pi)
# ax.axhline(0)
# ax.axhline(0)
# ax.set_xlim(-0.25,0.25)
# ax.set_ylim(-25,25)

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.plot(area*energies_stadium**2/4/pi, staircase_stadium-staircase_stadium[len(staircase_stadium)//2]*np.ones(len(staircase_stadium)))
ax.axline((0,0), slope = 1,ls = '--', c = 'r')
#ax.set_xlim(0,160)
#ax.set_ylim(-150,150)

In [ ]:
def spacing_ratios_distribution(spectrum, nbins = 41):
    
    spacings = (np.sort(spectrum)-np.roll(np.sort(spectrum),1))[1:]
    ratios = (spacings/np.roll(spacings,1))[1:]
    ratios = np.where(ratios<1, ratios, 1/ratios)
    histogram, bin_edges = np.histogram(ratios, range = (0,1), bins = nbins, density = True )
    
    return histogram, bin_edges

nbins_spacing = 15
ratios_distribution_stadium, bin_edges_stadium = spacing_ratios_distribution(spectrum_stadium, nbins = nbins_spacing)
#ratios_distribution_stadium, bin_edges_stadium = spacing_ratios_distribution(spectrum_stadium[:len(spectrum_stadium)//4], nbins = nbins_spacing)

x_stadium = (bin_edges_stadium[1:]+bin_edges_stadium[:-1])/2
dx_stadium = (bin_edges_stadium[1]-bin_edges_stadium[0])
analytical_gue = (81*np.sqrt(3)/(2*pi))*(x_stadium+x_stadium**2)**2/(1+x_stadium+x_stadium**2)**4

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.bar(x_stadium,ratios_distribution_stadium,width = dx_stadium)
ax.plot(x_stadium,analytical_gue, c = 'orange')
ax.set_ylabel(r"$P(r)$ $units$",fontsize = 18)
ax.set_xlabel(r"$r$ $units$",fontsize = 18)

In [ ]:
fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(111)
ax.scatter(x_stadium,ratios_distribution_stadium)
ax.scatter(x_stadium,ratios_distribution_ellipse)
ax.plot(x_stadium,analytical_gue, c = 'orange')
ax.set_ylabel(r"$P(r)$ $units$",fontsize = 18)
ax.set_xlabel(r"$r$ $units$",fontsize = 18)